Imports que importam :) 

In [46]:
import numpy as np
import math

import pandas

In [47]:
from random import *
cutpart = lambda x: (int((x+1) * np.random.normal(scale=0.2,loc=0.5)) )% x

In [48]:
va = [x for x in range(1,11)]
vb = [x for x in range(10,0,-1)]
print("va:\n{}\nvb:\n{}".format(va,vb))

va:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
vb:
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


<h1>Genetic Algorithm for QAP solving()</h1>

In [49]:
def crossover(va,vb):
    cut = cutpart(len(va))
    print("cut: {}".format(cut))
    new_va = va[:cut]+vb[cut:]
    new_vb = vb[:cut]+va[cut:]
    print("new va:\n{}\nnew vb:\n{}".format(new_va,new_vb))
    #encontrar repetidos (x) e suas repectivas posicoes (i)
    repeated_a = [(x,i) for x,i in zip(new_va,range(len(va))) if new_va.count(x) > 1]
    repeated_b = [(x,i) for x,i in zip(new_vb,range(len(vb))) if new_vb.count(x) > 1]
    rn = int(len(repeated_a)/2)
    print("rn:{}".format(rn))
    for i in range(rn):
        print("switch {} / {}".format(i,rn))
        new_va[repeated_a[i][1]] = repeated_b[i+ (0 if cut >5 else rn)][0]	#se trocar sempre o primeiro 
        new_vb[repeated_b[i][1]] = repeated_a[i+ (0 if cut >5 else rn)][0]	#repetido cortes acima de 5 tendem
    print("repeated a: {}".format(repeated_a[:int(len(repeated_a))]))		#a repertir os valores dos pais
    print("repeated b: {}".format(repeated_b[:int(len(repeated_b))]))
    return(new_va,new_vb)

In [50]:
new_va,new_vb = crossover(va,vb)#[1,5,3,4,6,10,7,9,8,2])
print("new va:\n{}\nnew vb:\n{}".format(new_va,new_vb))

cut: 4
new va:
[1, 2, 3, 4, 6, 5, 4, 3, 2, 1]
new vb:
[10, 9, 8, 7, 5, 6, 7, 8, 9, 10]
rn:4
switch 0 / 4
switch 1 / 4
switch 2 / 4
switch 3 / 4
repeated a: [(1, 0), (2, 1), (3, 2), (4, 3), (4, 6), (3, 7), (2, 8), (1, 9)]
repeated b: [(10, 0), (9, 1), (8, 2), (7, 3), (7, 6), (8, 7), (9, 8), (10, 9)]
new va:
[7, 8, 9, 10, 6, 5, 4, 3, 2, 1]
new vb:
[4, 3, 2, 1, 5, 6, 7, 8, 9, 10]


In [51]:
def inverse(vec):
    inf = int(random()*len(vec))
    part = list(vec[inf:])
    part.reverse()
    return vec[:inf]+part

In [52]:
def swap (vec):
    n_vec = list(vec)
    p1 = int(random()*len(vec))
    p2 = int(random()*len(vec))
    v1 = vec[p1]
    v2 = vec[p2]
    print("swap {} and {}".format(v1,v2))
    n_vec[p1] = v2
    n_vec[p2] = v1
    return n_vec

In [53]:
def insertion(vec):
    inf = int(random()*len(vec))
    part = vec[inf:]
    part = part[1:] + part[:1]
    print(part)
    return vec[:inf]+part

In [54]:
def mutation(vec,p):#percentual
    if random() > p:
        return vec
    typ = random()
    if(typ < 1/3):
        print("inverse")
        vec = inverse(vec)
    elif(typ < 2/3):
        print("swap")
        vec = swap(vec)
    else:
        print("insertion")
        vec = insertion(vec)
    return vec
    
#mutation(va,0.7)
print("Original:")
print(va)
print("Inverse:")
print(inverse(va))
print("Swap:")
print(swap(va))
print("Insertion")
print(insertion(va))

Original:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Inverse:
[1, 2, 3, 4, 5, 6, 10, 9, 8, 7]
Swap:
swap 1 and 4
[4, 2, 3, 1, 5, 6, 7, 8, 9, 10]
Insertion
[10, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 9]


In [55]:
mutation(va,0.7)

swap
swap 8 and 10


[1, 2, 3, 4, 5, 6, 7, 10, 9, 8]

<h1>Calculo de Coorelação:</h1>

In [56]:
def corr(x,y,p=False):
    # n
    n = len(x)
#Somatórios:
    #superiores
    xysum = np.dot(x,y) #soma dos x*y
    xsum = sum(x)
    ysum = sum(y)
    #inferiores
    #soma dos quadrados
    x2sum = np.dot(x,x) #multiplica entrada a entrada dos vetores x e ele mesmo e soma
    y2sum = sum([v*v for v in y]) #forma na unha de fazer o mesmo
    #quadrado das somas:
    xsum2 = xsum**2
    ysum2 = ysum**2
    #parte superior:
    sup = (xysum - xsum*ysum/n)
    #parte inferior:
    infe = math.sqrt ( (x2sum - xsum2/n) * (y2sum - ysum2/n) )
    corresult = sup/infe
    if(p):
        print("Taxa de correlação entre X e Y é:")
        print("{}\n".format(sup)+len(str(sup))*"-"+" =  {}".format(corresult)+"\n{}".format(infe))
    return corresult

<h2>Teste Da Correlação</h2>
<p>Para calcular a correlação entre os valores de duas variáveis basta entrar com dois vetores com os valores a serem comparados:
corr(vec1,vec2)</p>

In [57]:
gerax = lambda base,n,dif: [base + x*dif for x in range(n)] # modo preguiçoso de gerar n números espaçados em 0.5
#Carregar valores(teste):
x = gerax(2.5, 6,0.5)
y = [0.4,0.4,0.3,0.2,0.2,0.1]
corr(x,y,True)

Taxa de correlação entre X e Y é:
-0.5499999999999998
------------------- =  -0.9710083124552239
0.5664215155988811


-0.9710083124552239

<h1>Carregamento de Dados</h1>
<p>Nesta sessão do código é feita a leitura dos arquivos .csv</p>

In [58]:
#!/usr/bin/python

from pathlib import Path

directory = './nodes_links/'
allnodes = dict()
alllinks = dict()
names = list()
for path in Path(directory).iterdir():
    url = str(path)
    netname = path.name.split('_')[0]
    if(path.name.count('nodes')==1):
        p = allnodes
    else:
        p = alllinks
    try:
        item = pandas.read_csv(url)
        p.update( {netname : item })
    #    print(url+" lido")
    except:
        try:
            #mudar o encoding para os caracteres
            item = pandas.read_csv(url,encoding="iso-8859-1")
            p.update( {netname : item })
        #    print(url+" lido")
        except:
            print("!!! não foi possivel ler > "+name+" < !!!")
            pass
    if names.count(netname) == 0:
        names.append(netname) 


<p>Todos os links e nós ficam guardados nos dicts alllinks e allnodes, e para serem lidos basta passar o prefixo dos arquivos originais entre colchetes. Estes prefixos são os nomes das nossas redes e ficam salvos no vetor names.</p>

In [59]:
print(names)

['canarie', 'italy', 'vbns', 'loni', 'germany', 'geant2', 'arpanet', 'cesnet', 'newnet', 'rnpBrazil', 'pionier', 'spain', 'renater', 'lambdaRailUsa', 'metrona', 'arnes', 'bren', 'portugal', 'usaGde', 'sanet', 'nsfnet', 'austria', 'viaDatacenterNet', 'coxUsa', 'mzima', 'scteste', 'eon', 'memorexEurope', 'deutschTelecom', 'internet2Usa', 'OmnicomEurope']


<p>Abaixo temos o exemplo do carregamento da rede cesnet:</p>

In [60]:
netlinks = alllinks['cesnet']
netnodes = allnodes['cesnet']

<p>Nos arquivos de nós temos no campo Id todos os nomes dos nós da rede, e nos links a origem e destino dos enlaces, como podemos ver abaixo:</p>

In [61]:
#u = pandas.read_csv("./nodes_links/renater_links.csv")
a = netnodes['Id'].values.tolist()
print("nodes:\n{}".format(a))
print("links:\n{}".format(netlinks))

nodes:
['Praga', 'Usti', 'Liberec', 'Hradec', 'Olomouc', 'Ostrava', 'Zlin', 'Brno', 'Jihlava', 'Budweiss', 'Pilsen', 'Pardubice']
links:
         From         To      Length  Capacity  Cost
0       Praga       Usti   44.320945        50     1
1        Usti    Liberec   57.122562        50     1
2     Liberec      Praga   57.111162        50     1
3       Praga     Pilsen  114.930257        50     1
4      Pilsen   Budweiss  123.277291        50     1
5    Budweiss      Praga  169.402099        50     1
6       Praga  Pardubice   72.350748        50     1
7   Pardubice     Hradec   69.868247        50     1
8      Hradec      Praga   91.221450        50     1
9      Hradec    Liberec   64.755029        50     1
10   Budweiss    Jihlava  106.688652        50     1
11    Jihlava       Brno   74.878684        50     1
12       Brno   Budweiss  163.463112        50     1
13    Olomouc     Hradec  136.245096        50     1
14    Olomouc       Zlin   30.591459        50     1
15       Zlin  

<h2>Gerando as matrizes de Adjacência</h2>
<p>Para montar as matrizes de adjacência consideramos a identificação numérica dos nós conforme a posição de cada um no vetor dos nós. Depois, com a ajuda da comparação de atributos da biblioteca pandas verificamos todas as origens e quais seus destinos, e assim preenchemos a matriz de adjacência.</p>
<p>A matriz de adjacência é bidirecional, para facilitar o uso com a implementação com qualquer algorítmo de busca a construção é nas diagonáis superiores e inferiores.</p>

In [62]:
def monta_matriz(netnodes,netlinks):
    nodes = netnodes['Id'].values.tolist()
    matrix = np.matrix(np.zeros( (len(nodes),len(nodes)),dtype=np.int ) )
    for n in nodes:
        l = netlinks.loc[netlinks['From']==n,['To']].values
        indexes = [nodes.index(x) for x in l]
        if(len(indexes) > 0):
            matrix[nodes.index(n),indexes] = 1
            matrix[indexes,nodes.index(n)] = 1 #bidirecional
    return matrix

matrix = monta_matriz(netnodes,netlinks)

print (matrix)

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


<p>Abaixo temos a construção das listas que representam os canais de comunicação dos enlaces. Estes canais são representados por 0's e 1's guardadas em listas salvas em dicts que são acessados pela tupla que representa o origem e destino.</p>

In [63]:
def criar_canais(mtx,ncanais):
    canais = dict()
#     mtxc = np.matrix(np.zeros(mtx.shape), dtype=list)
    for i in range(len(mtx)):
        for j in range(len(mtx)):
            if(i<j):
                if(mtx[i,j] == 1):
                    lista = [1]*ncanais #1 livre, 0 ocupado
                    canais.update({(i,j):lista})
                    canais.update({(j,i):lista})#mesma instancia para as duas direções
    return canais
#                    mtxc[i,j]= [1]*ncanais
#                    mtxc[j,i]= mtxc[i,j]
#                 else:
#                     mtxc[i,j]= None
#     return mtxc

<p>Como os canais são bidirecionais os mesmos canais são usados nas duas direções, ou seja, do nó 0 para o nó 9 e do nó 9 para o nó 0 a mesma instância do objeto lista é utilizado. Assim se o canal 0 é ocupado em uma direção ele também não estará disponível no inverso, como podemos ver abaixo:</p> 

In [64]:
canais_rede = criar_canais(matrix,len(netnodes))
canais_rede[(9,0)][0]=0 # ocupar canal 0 do enlace
print (canais_rede[0,9])
canais_rede[(9,0)][0]=1 # desocupar canal 0 do enlace
print (canais_rede[0,9])

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<p>Excluir a diagonal inferior(caso necessário):</p>

In [65]:
def diagonalizar(mtx):
    nova = np.matrix(np.zeros(mtx.shape), dtype=int)
    for i in range(len(mtx)):
       # print(l)
        for j in range(len(mtx)):
            if(i<j):
                nova[i,j] = mtx[i,j]
    return nova
# print(diagonalizar(matrix))

<h2>Calculo e Incrementação do grau</h2>
<p>Para calcular o grau médio basta calcular a quantidade de elementos não nulos na linha ou coluna de cada nó, e depois dividir pelo núme de nós.</p>
<p><i>Obs. Isso se aplica quando a matriz é bidirecional e as diagonais superior e inferior são não nulas.</i></p>

In [66]:
def calc_grau(matrix):
    soma=0
    for l in range(len(matrix)):
        for c in range(len(matrix)):
            if matrix[l,c] != 0:
                soma += 1
    return soma/(len(matrix))

In [67]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 3.1666666666666665


<p>Para aumentar o grau da nossa topologia basta inserir novos enlaces de forma aleatória na matriz de adjacência. Para isso adiciona-se enlaces até o grafo atingir o grau desejado</p>

In [68]:
def aumentar_grau(mtx,inc_grau): #nodos = 6, p/incrementar 1 grau adicionamos 6 enlaces, para inc 0.5 add 3
    grau = calc_grau(mtx)
    mind = len(mtx)-1
    n_grau = grau
    while n_grau < mind and n_grau < grau + inc_grau:
        x,y=(None,None)
        while (x==None and y==None):
            x,y = (randint(0,mind), randint(0,mind))
            if x==y or mtx[x,y]==1:
                x,y=(None,None)
        mtx[x,y]=1
        mtx[y,x]=1 #bijetora...
        n_grau=calc_grau(mtx)

<p><b>Antes:</b></p>

In [69]:
print(matrix)

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


In [70]:
aumentar_grau(matrix,5)

<p><b>Depois:</b></p>

In [71]:
print(matrix)

[[0 1 1 1 1 1 1 1 0 1 1 1]
 [1 0 1 0 1 1 0 1 1 0 1 1]
 [1 1 0 1 0 1 0 0 0 0 0 1]
 [1 0 1 0 1 1 1 1 0 1 0 1]
 [1 1 0 1 0 1 1 1 0 1 1 1]
 [1 1 1 1 1 0 1 1 1 1 1 1]
 [1 0 0 1 1 1 0 1 1 1 1 1]
 [1 1 0 1 1 1 1 0 1 1 1 0]
 [0 1 0 0 0 1 1 1 0 1 0 0]
 [1 0 0 1 1 1 1 1 1 0 1 1]
 [1 1 0 0 1 1 1 1 0 1 0 0]
 [1 1 1 1 1 1 1 0 0 1 0 0]]


In [72]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 8.166666666666666


<h2>Construir todas as matrizes</h2>
<p>Nesta etapa geramos todas as matrizes à partir das informações tiradas dos arquivos nodes e links.

In [73]:
matrizes = dict()
#montar matrizes, canais, etc...
for x in names:
    netlinks = alllinks[x]
    netnodes = allnodes[x]
    matriz = monta_matriz(netnodes,netlinks)
    matrizes.update({x:matriz})
#     print("matriz {}:\n{}".format(x,matriz))

<p>Para acessar as matrizes basta passar o nome da rede entre []:</p>

In [74]:
mat=matrizes['cesnet']
print("matriz:\n")
print(mat)

matriz:

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


<h1>Dijkstra</h1>
<p>Implementação da busca pelo menor caminho</p>

In [75]:
#Matriz de Incidência
from math import inf

letra = lambda x:chr(ord('a')+x)

class BuscaMatriz():
    #matrizNova=list()
    def __init__(self,matrix):
        self.matrix=matrix
        self.custo=[inf]*matrix.size()
        self.pai=[-1]*matrix.size()
    def backtrack(self,destino):
        cam=list()
#         print(destino)
        pai=self.pai[destino]
        cam.append(destino)
        if(pai!=-1):
            cam = cam + self.backtrack(pai)
        return cam

class GraphMatrix():
    def __init__(self,first,matrix):#de certa forma isto não deveria estar aqui e desta forma
        self.actual=first
        self.matrix=matrix
        self.custo=0
        self.visited=[0]*len(matrix)
        self.visited[first]=1
#         for i,linha in enumerate(matrix):
#             print ("linha %d: %s"%(i,linha))
    def __str__(self):
        return 'teste'
    def visit(self,aresta,custo):
        if (self.visited[aresta]==1):
            return False
        self.visited[aresta]=1
        self.actual=aresta
        self.custo+=custo
        return True
    def get_actual(self):
        return self.actual
    def set_actual(self,actual):
        self.actual=actual
    def size(self):
        return len(self.matrix)
    def run(self,*reverse): #nem isto
#         print("Atualmente em %d com custo %d" % (self.actual,self.custo))
        vizinhos=self.get_vizinhos(self.actual)
#         print ("vizinhos:")
#         for vizinho in vizinhos:
#             print("numero %d custo %d" % vizinho)
        while len(vizinhos)>0:
            if reverse:
                v=vizinhos.pop()
            else:
                v=vizinhos.pop(0)
            if self.visit(*v) == True:
                return True
        return False

class AdjMatrix(GraphMatrix):
    def __init__(self,*args):
        super(AdjMatrix,self).__init__(*args)
    def get_vizinhos(self,vert):
#         print(self.matrix)
        vizinhos=[(i,ar) for i,ar in enumerate(self.matrix[vert]) if ar>0 ]
        vizinhos.sort(key=lambda tupla:tupla[1])
        return vizinhos

In [76]:
#import mtx /\
import sys
from math import inf
class Dijkstra(BuscaMatriz):
    show=lambda self:print("Custo: %s\nPai: %s\nFila %s\nAtual: %d" % (self.custo,self.pai,self.fila,self.matrix.get_actual()))
    def __init__(self,matrix):
        super(Dijkstra,self).__init__(matrix)
        self.fila=list(range(0,matrix.size()))
        self.fila.remove(self.matrix.get_actual())

    def explora(self):
        index=self.matrix.get_actual()
        if(self.custo[index]==inf):#caso seja desconexo
            self.custo[index]=0
        for vizinho in self.matrix.get_vizinhos(index):
            (vert,value)=vizinho
#             self.show()
            nvalue=value+self.custo[index]
            if nvalue < self.custo[vert]:
#                 print("Novo valor para %d: %d"%(vert,nvalue))
                self.atualiza(vert,nvalue)
    def gen_fila(self):
        for x in self.fila:
            custo=self.custo[x]
            yield (custo,x)
    def end(self):
        return self.fila==[]
    def atualiza(self,filho,valor):
        self.custo[filho]=valor
        self.pai[filho]=self.matrix.get_actual()
    def choose(self):
        fila=list()
        for a in self.gen_fila():
            fila.append(a)
        fila.sort()
#         print("\nFila em ordem de peso: %s"%(fila))
        (peso,actual)=fila.pop(0)
        
        self.matrix.set_actual(actual)
        self.fila.remove(actual)

<h2>Calcular as rotas</h2>
<p>Ao explorar a matriz verificamos os menores custos para comunicação entre todos os nós.</p>

In [77]:
def explorar_matrix(mtx):
    caminhos = dict()
    lmtx = mtx.tolist()
    for i in range(len(mtx)):
        for j in range(len(mtx)):
#             print("==========constroi:{}==================".format(i))
#             if(i<j): #neste passo apenas as entradas da matriz superior serão passadas como caminhos
            if i != j:
#                 print("======================================".format(i))
                di=Dijkstra(AdjMatrix(i,lmtx))
#                 print("============={}-{}====================".format(i,j))
                di.explora()
                while(di.end()==False):
                    di.explora()
                    di.choose()
             #   caminhos.update({(i,j):di.backtrack(j)})
#                 print("========backtrack={}-{}===============".format(i,j))
#                 di.backtrack(j)
                caminhos.update({(j,i):di.backtrack(j)})
    return caminhos

In [78]:
matrix = np.matrix(matrizes.get('cesnet'))#nova instancia
caminhos = explorar_matrix(matrix)

In [79]:
print(matrix)

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


<p>O indice de wiener é expresso pelo comprimento de todos os caminhos curtos descobertos</p>

In [80]:
def wiener(caminhos,mtx):
    custo = 0
    for i in range(len(mtx)):
        for j in range(len(mtx)):
            if(i<j):
                custo+= (len(caminhos[i,j]) -1)
    return custo

In [81]:
print(caminhos)
print("Wiener: {}".format(wiener(caminhos,matrix)))

{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 0], (4, 0): [4, 3, 0], (5, 0): [5, 4, 3, 0], (6, 0): [6, 4, 3, 0], (7, 0): [7, 9, 0], (8, 0): [8, 9, 0], (9, 0): [9, 0], (10, 0): [10, 0], (11, 0): [11, 0], (0, 1): [0, 1], (2, 1): [2, 1], (3, 1): [3, 0, 1], (4, 1): [4, 3, 0, 1], (5, 1): [5, 4, 3, 0, 1], (6, 1): [6, 4, 3, 0, 1], (7, 1): [7, 9, 0, 1], (8, 1): [8, 9, 0, 1], (9, 1): [9, 0, 1], (10, 1): [10, 0, 1], (11, 1): [11, 0, 1], (0, 2): [0, 2], (1, 2): [1, 2], (3, 2): [3, 2], (4, 2): [4, 3, 2], (5, 2): [5, 4, 3, 2], (6, 2): [6, 4, 3, 2], (7, 2): [7, 4, 3, 2], (8, 2): [8, 9, 0, 2], (9, 2): [9, 0, 2], (10, 2): [10, 0, 2], (11, 2): [11, 0, 2], (0, 3): [0, 3], (1, 3): [1, 0, 3], (2, 3): [2, 3], (4, 3): [4, 3], (5, 3): [5, 4, 3], (6, 3): [6, 4, 3], (7, 3): [7, 4, 3], (8, 3): [8, 7, 4, 3], (9, 3): [9, 0, 3], (10, 3): [10, 0, 3], (11, 3): [11, 3], (0, 4): [0, 3, 4], (1, 4): [1, 0, 3, 4], (2, 4): [2, 3, 4], (3, 4): [3, 4], (5, 4): [5, 4], (6, 4): [6, 4], (7, 4): [7, 4], (8, 4): [8, 7, 4], (9, 4):

In [82]:
rotear=lambda caminhos,origem,destino: [par for par in zip(caminhos[(origem,destino)],caminhos[(origem,destino)][1:]) ]

In [83]:
print(calc_grau(matrix))
print("caminho:\n{}".format(caminhos[(2,5)]))
print("enlaces:\n{}".format(rotear(caminhos,2,5)))

3.1666666666666665
caminho:
[2, 3, 4, 5]
enlaces:
[(2, 3), (3, 4), (4, 5)]


In [84]:
aumentar_grau(matrix,4)
print (matrix)
print (calc_grau(matrix))

[[0 1 1 1 1 0 1 1 1 1 1 1]
 [1 0 1 0 1 1 0 0 1 0 1 1]
 [1 1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 0 1 0 1 1 1 1 0 1]
 [1 1 0 1 0 1 1 1 1 0 1 0]
 [0 1 1 0 1 0 1 1 0 0 0 0]
 [1 0 1 1 1 1 0 1 1 1 0 0]
 [1 0 0 1 1 1 1 0 1 1 1 0]
 [1 1 0 1 1 0 1 1 0 1 0 0]
 [1 0 0 1 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 0 0 1 0 1 0 1]
 [1 1 1 1 0 0 0 0 0 0 1 0]]
7.166666666666667


In [85]:
print(caminhos)
caminhos = explorar_matrix(matrix)

print("Wiener: {}".format(wiener(caminhos,matrix)))

{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 0], (4, 0): [4, 3, 0], (5, 0): [5, 4, 3, 0], (6, 0): [6, 4, 3, 0], (7, 0): [7, 9, 0], (8, 0): [8, 9, 0], (9, 0): [9, 0], (10, 0): [10, 0], (11, 0): [11, 0], (0, 1): [0, 1], (2, 1): [2, 1], (3, 1): [3, 0, 1], (4, 1): [4, 3, 0, 1], (5, 1): [5, 4, 3, 0, 1], (6, 1): [6, 4, 3, 0, 1], (7, 1): [7, 9, 0, 1], (8, 1): [8, 9, 0, 1], (9, 1): [9, 0, 1], (10, 1): [10, 0, 1], (11, 1): [11, 0, 1], (0, 2): [0, 2], (1, 2): [1, 2], (3, 2): [3, 2], (4, 2): [4, 3, 2], (5, 2): [5, 4, 3, 2], (6, 2): [6, 4, 3, 2], (7, 2): [7, 4, 3, 2], (8, 2): [8, 9, 0, 2], (9, 2): [9, 0, 2], (10, 2): [10, 0, 2], (11, 2): [11, 0, 2], (0, 3): [0, 3], (1, 3): [1, 0, 3], (2, 3): [2, 3], (4, 3): [4, 3], (5, 3): [5, 4, 3], (6, 3): [6, 4, 3], (7, 3): [7, 4, 3], (8, 3): [8, 7, 4, 3], (9, 3): [9, 0, 3], (10, 3): [10, 0, 3], (11, 3): [11, 3], (0, 4): [0, 3, 4], (1, 4): [1, 0, 3, 4], (2, 4): [2, 3, 4], (3, 4): [3, 4], (5, 4): [5, 4], (6, 4): [6, 4], (7, 4): [7, 4], (8, 4): [8, 7, 4], (9, 4):

In [86]:
print("caminhos:\n{}".format(caminhos[(2,5)]))
print("enlaces:\n{}".format([par for par in zip(caminhos[(2,5)],caminhos[(2,5)][1:]) ]))

caminhos:
[2, 5]
enlaces:
[(2, 5)]


<h1>Executar os testes e simulação</h1>
<p>Por fim, executa-se diversas iterações contando o número bloqueios em cada iteração, a cada iteração o grau médio é aumentado. Por fim, calcula-se a correlação entre a taxa de bloqueio e o grau médio.</p>

In [183]:
class sim_rede():
    def __init__(self,nome,matrix_np,ncanais=2):
        self.nome = nome
        self.matrix = np.matrix(matrix_np) #criar uma nova instância, cópia das originais
        if(ncanais<1):
            ncanais = int(len(self.matrix)/3)
        self.ncanais = ncanais
        self.canais = criar_canais(self.matrix,ncanais)
        self.caminhos = explorar_matrix(self.matrix)
        self.bloqueios = 0
#         self.demandas = np.matrix(np.zeros(matrix_np.shape),dtype=int)
        self.sum_demandas = 0
#         for k in self.caminhos:
#             print("{}:{}".format(k,self.caminhos[k]))
    def atualiza_rede(self):
        self.caminhos = explorar_matrix(self.matrix)
        self.canais = criar_canais(self.matrix,self.ncanais) # para resetar todos eles
#     def aumenta_grau(self,valor):
#         aumentar_grau(self.matrix,valor)
#         self.atualiza_rede()
    def calcula_grau(self):
        return calc_grau(self.matrix)
    def calc_enlaces(self,origem,destino):
        caminho = self.caminhos[(origem,destino)]
        rota = [par for par in zip(caminho,caminho[1:]) ]
        print("canais:{}, origem:{}, destino:{}, rota:{}, caminho: {}".format(self.ncanais,origem,destino,rota,caminho))
        return rota
    def testa_canal(self,x,y):#nó por parametro
        return sum(self.canais[no])
    def conectar(self,origem,destino):
        ok=1
        enlaces = self.calc_enlaces(origem,destino)

        caminho = self.caminhos[(origem,destino)]
        
        for canal in range(self.ncanais): #canal 0..n
            for i,j in enlaces: #nó do caminho
#                 print(self.canais)
                ok = ok*self.canais[(i,j)][canal]
            self.sum_demandas+=1
            if ok == 1:
                for no in caminho: #no do caminho
                    self.canais[(j,i)][canal] = 0
#                     if i > j:
#                         self.demandas[(j,i)]+=1
#                     else:
#                         self.demandas[(i,j)]+=1    

                return (True,caminho)
            else:
                ok=1
        print("bloqueio ({}, {})".format(origem,destino))
        self.bloqueios += 1
        return (False,caminho)
    def wiener(self):
        return wiener(self.caminhos,self.matrix)

<p>Exemplo de uma rede pequena, que com seu grau médio original provocava 4 bloqueios, tem seu grau médio aumentado 2 vezes seguidas.</p>

In [184]:
n='scteste'
matrix = np.matrix(matrizes.get(n)) #cópia instanciada
for i in range(3):
    print("\n===========iteracao ({})============\n".format(i))
    rede = sim_rede(n,matrix)
    print("wiener antes: {}".format(rede.wiener()))
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if(i<j):
    #             print(rede.caminhos[i,j])
                rede.conectar(i,j)
    aumentar_grau(matrix,1)
    print("demandas:{}, rede:{}, bloqueios:{}, wiener: {}".format(rede.sum_demandas,n,rede.bloqueios,rede.wiener()))
#     print("demandas(total={}):\n{}".format(rede.sum_demandas,rede.demandas))
    print("matriz:\n{}".format(matrix))


===========iteracao (0)============

wiener antes: 25
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 1), (1, 3)], caminho: [0, 1, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 3), (3, 4)], caminho: [0, 1, 3, 4]
bloqueio (0, 4)
canais:2, origem:0, destino:5, rota:[(0, 2), (2, 5)], caminho: [0, 2, 5]
canais:2, origem:1, destino:2, rota:[(1, 0), (0, 2)], caminho: [1, 0, 2]
canais:2, origem:1, destino:3, rota:[(1, 3)], caminho: [1, 3]
canais:2, origem:1, destino:4, rota:[(1, 3), (3, 4)], caminho: [1, 3, 4]
bloqueio (1, 4)
canais:2, origem:1, destino:5, rota:[(1, 0), (0, 2), (2, 5)], caminho: [1, 0, 2, 5]
bloqueio (1, 5)
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 3), (3, 4)], caminho: [2, 3, 4]
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
canais:2, origem:3, destino:4, rota:[(3, 4)], caminho:

<h1>Simulação e cálculo de correlação</h1>
<p>Por fim simula-se o uso da rede</p>

In [193]:
iteracoes = 5
ind_wiener = dict()
bloqs = dict()
grau = dict()
demandas = dict()
inc_grau = 1
copias = dict()

for n in names:
    ind_wiener.update({n:[]})
    bloqs.update({n:[]})
    grau.update({n:[]})
    copias.update({n:np.matrix(matrizes.get(n))}) 
    demandas.update({n:[]})

for n in names:#[n for n in names if n!='rnpBrazil' and n!='bren' and n!='viaDatacenterNet']:
    for it in range(iteracoes):
#         n='cesnet'
        rede = sim_rede(n,copias[n])
        print(matrizes.get(n))
        print("rede: {}".format(n))
        for l in range(4):
            for i in range(len(matrix)):
                for j in range(len(matrix)):
                    if(i<j):
                        rede.conectar(i,j)
        grau[n] += [rede.calcula_grau()]
        bloqs[n] += [rede.bloqueios]
        ind_wiener[n] += [rede.wiener()]
        demandas[n] += [rede.sum_demandas]
        print("rede:{}, grau:{}, bloqueios:{}, wiener: {}".format(n,grau[n],rede.bloqueios,ind_wiener[n][it]))
        if calc_grau(copias[n]) == (len(copias[n])-1):
            break
        aumentar_grau(copias[n],inc_grau)

[[0 1 1 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]]
rede: OmnicomEurope
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 3)], caminho: [0, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 2), (2, 5)], caminho: [0, 2, 5]
canais:2, origem:1, destino:2, rota:[(1, 2)], caminho: [1, 2]
canais:2, origem:1, destino:3, rota:[(1, 0), (0, 3)], caminho: [1, 0, 3]
canais:2, origem:1, destino:4, rota:[(1, 4)], caminho: [1, 4]
canais:2, origem:1, destino:5, rota:[(1, 2), (2, 5)], caminho: [1, 2, 5]
bloqueio (1, 5)
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 1), (1, 4)], caminho: [2, 1, 4]
bloqueio (2, 4)
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
canais:2, origem:3, destino:4

[[0 1 1 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]]
rede: OmnicomEurope
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 3)], caminho: [0, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 2), (2, 5)], caminho: [0, 2, 5]
canais:2, origem:1, destino:2, rota:[(1, 2)], caminho: [1, 2]
canais:2, origem:1, destino:3, rota:[(1, 0), (0, 3)], caminho: [1, 0, 3]
canais:2, origem:1, destino:4, rota:[(1, 4)], caminho: [1, 4]
canais:2, origem:1, destino:5, rota:[(1, 2), (2, 5)], caminho: [1, 2, 5]
bloqueio (1, 5)
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 1), (1, 4)], caminho: [2, 1, 4]
bloqueio (2, 4)
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
canais:2, origem:3, destino:4

[[0 1 1 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]]
rede: OmnicomEurope
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 3)], caminho: [0, 3]
canais:2, origem:0, destino:4, rota:[(0, 4)], caminho: [0, 4]
canais:2, origem:0, destino:5, rota:[(0, 2), (2, 5)], caminho: [0, 2, 5]
canais:2, origem:1, destino:2, rota:[(1, 2)], caminho: [1, 2]
canais:2, origem:1, destino:3, rota:[(1, 3)], caminho: [1, 3]
canais:2, origem:1, destino:4, rota:[(1, 4)], caminho: [1, 4]
canais:2, origem:1, destino:5, rota:[(1, 2), (2, 5)], caminho: [1, 2, 5]
bloqueio (1, 5)
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 0), (0, 4)], caminho: [2, 0, 4]
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
canais:2, origem:3, destino:4, rota:[(3, 0), (0, 4)], caminho: [3, 

[[0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]]
rede: arnes
canais:2, origem:0, destino:1, rota:[(0, 7), (7, 1)], caminho: [0, 7, 1]
canais:2, origem:0, destino:2, rota:[(0, 7), (7, 6), (6, 2)], caminho: [0, 7, 6, 2]
canais:2, origem:0, destino:3, rota:[(0, 7), (7, 1), (1, 3)], caminho: [0, 7, 1, 3]
canais:2, origem:0, destino:4, rota:[(0, 7), (7, 4)], caminho: [0, 7, 4]
canais:2, origem:0, destino:5, rota:[(0, 7),

[[0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]]
rede: arnes
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 10), (10, 2)], caminho: [0, 10, 2]
canais:2, origem:0, destino:3, rota:[(0, 1), (1, 3)], caminho: [0, 1, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 1), (1, 5)], caminho: [0, 1, 5]
c

...

[[0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 1 1 0 1 0 1 1 0 1 0 1]
 [0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]]
rede: spain
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 3)], caminho: [0, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 8), (8, 5)], caminho: [0, 8, 5]
canais:2, origem:1, destin

[[0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
rede: usaGde
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 1), (1, 3)], caminho: [0, 1, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 2), (2, 5)], caminho: [0, 2, 5]
canais:2, origem:1, destino:2, rota:[(1, 0), (0, 2)], caminho: [1, 0, 2]
canais:2, origem:1, destino:3, rota:[(1, 3)], caminho: [1, 3]
canais:2, origem:1, destino:4, rota:[(1, 4)], caminho: [1, 4]
canais:2, origem:1, destino:5, rota:[(1, 0), (0, 2), (2, 5)], caminho: [1, 0, 2, 5]
bloqueio (1, 5)
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 0), (0, 1), (1, 4)], caminho: [2, 0, 1, 4]
bloqueio (2, 4)
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
can

[[0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
rede: usaGde
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 1), (1, 3)], caminho: [0, 1, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 2), (2, 5)], caminho: [0, 2, 5]
canais:2, origem:1, destino:2, rota:[(1, 0), (0, 2)], caminho: [1, 0, 2]
canais:2, origem:1, destino:3, rota:[(1, 3)], caminho: [1, 3]
canais:2, origem:1, destino:4, rota:[(1, 4)], caminho: [1, 4]
canais:2, origem:1, destino:5, rota:[(1, 0), (0, 2), (2, 5)], caminho: [1, 0, 2, 5]
bloqueio (1, 5)
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 95), (95, 4)], caminho: [2, 95, 4]
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
canais:2, origem:3, destino

[[0 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0]
 [1 1 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 1 0 1]
 [0 0 0 0 0 1 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 0 0 1 0 1]
 [0 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 1 1 1 0 1 0]]
rede: vbns
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 1), (1, 2)], caminho: [0, 1, 2]
canais:2, origem:0, destino:3, rota:[(0, 3)], caminho: [0, 3]
canais:2, origem:0, destino:4, rota:[(0, 3), (3, 4)], caminho: [0, 3, 4]
canais:2, origem:0, destino:5, rota:[(0, 1), (1, 2), (2, 5)], caminho: [0, 1, 2, 5]
bloqueio (0, 5)
canais:2, origem:1, destino:2, rota:[(1, 2)], caminho: [1, 2]
canais:2, origem:1, destino:3, rota:[(1, 3)], caminho: [1, 3]
canais:2, origem:1, destino:4, rota:[(1, 3), (3, 4)], caminho: [1, 3, 4]
bloqueio (1, 4)
canais:2, origem:1, destino:5, rota:[(1, 2), (2, 5)], caminho: [1, 2, 5]
bloqueio

[[0 1 1 1 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 0]
 [1 1 0 1 1 1 0 0 0]
 [1 0 1 0 0 0 1 0 0]
 [0 0 1 0 0 1 0 0 0]
 [0 0 1 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 1]
 [1 0 0 0 0 0 0 1 0]]
rede: viaDatacenterNet
canais:2, origem:0, destino:1, rota:[(0, 1)], caminho: [0, 1]
canais:2, origem:0, destino:2, rota:[(0, 2)], caminho: [0, 2]
canais:2, origem:0, destino:3, rota:[(0, 3)], caminho: [0, 3]
canais:2, origem:0, destino:4, rota:[(0, 1), (1, 4)], caminho: [0, 1, 4]
canais:2, origem:0, destino:5, rota:[(0, 1), (1, 5)], caminho: [0, 1, 5]
canais:2, origem:1, destino:2, rota:[(1, 2)], caminho: [1, 2]
canais:2, origem:1, destino:3, rota:[(1, 3)], caminho: [1, 3]
canais:2, origem:1, destino:4, rota:[(1, 4)], caminho: [1, 4]
canais:2, origem:1, destino:5, rota:[(1, 5)], caminho: [1, 5]
canais:2, origem:2, destino:3, rota:[(2, 3)], caminho: [2, 3]
canais:2, origem:2, destino:4, rota:[(2, 4)], caminho: [2, 4]
canais:2, origem:2, destino:5, rota:[(2, 5)], caminho: [2, 5]
canais:2, origem:3, d

In [195]:
names.sort()
for n in names:
    print ("\n{}({}):".format(n,len(matrizes[n])))
    print("wiener	demand	Blocks	grau")
    tb = []
    for a,b,c,d in zip(ind_wiener[n],demandas[n],bloqs[n],grau[n]):
        print ("{}	{}	{}	{}".format(a,b,c,d))
        tb.append(b/c)
    corr(tb,ind_wiener[n],True)
    corr(tb,grau[n],True)


OmnicomEurope(38):
wiener	demand	Blocks	grau
3166	112	44	2.8421052631578947
2009	112	44	3.8421052631578947
1685	112	44	4.842105263157895
1531	111	42	5.842105263157895
1437	110	40	6.842105263157895
Taxa de correlação entre X e Y é:
-150.45389610389975
------------------- =  -0.5863159955343905
256.608888807085
Taxa de correlação entre X e Y é:
0.5064935064935057
------------------ =  0.8804710999221848
0.5752528465025929

arnes(17):
wiener	demand	Blocks	grau
411	114	48	2.3529411764705883
309	114	47	3.411764705882353
267	112	44	4.470588235294118
240	110	40	5.529411764705882
219	111	42	6.588235294117647
Taxa de correlação entre X e Y é:
-40.03713042276877
------------------ =  -0.8564342287039839
46.748634140132104
Taxa de correlação entre X e Y é:
0.9107813338101138
------------------ =  0.8837595358936011
1.0305759619206711

arpanet(20):
wiener	demand	Blocks	grau
523	113	46	3.2
400	112	44	4.2
365	110	40	5.2
331	110	40	6.2
312	110	40	7.2
Taxa de correlação entre X e Y é:
-42.97055335968

In [92]:
print(matrizes['rnpBrazil'])
print(len(matrizes['rnpBrazil']))

[[0 1 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 1 0 0]
 [0 0 1 0 1 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 1]
 [1 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 1 0]]
10
